# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import math
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import optuna
from copy import deepcopy
from pyDeepInsight import ImageTransformer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.base import clone

from scipy.spatial.distance import cdist
from scipy.stats import ks_2samp
from scipy.optimize import minimize

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

## MAE

In [5]:
class Encoder(nn.Module):
    def __init__(self, img_channels=1, feature_dim=32, latent_dim=2):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(img_channels, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(32 * 2 * 2, feature_dim)

    def forward(self, x):
        x = self.relu(self.conv1(x))  # Output: (batch_size, 16, 8, 8)
        x = self.pool(x)              # Output: (batch_size, 16, 4, 4)
        x = self.relu(self.conv2(x))  # Output: (batch_size, 32, 4, 4)
        x = self.pool(x)              # Output: (batch_size, 32, 2, 2)
        x = x.view(x.size(0), -1)     # Flatten to (batch_size, 128)
        x = self.fc1(x)               # Output: (batch_size, feature_dim)
        return x

class Decoder(nn.Module):
    def __init__(self, img_channels=1, feature_dim=32):
        super(Decoder, self).__init__()
        self.fc2 = nn.Linear(feature_dim, 32 * 2 * 2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        self.deconv1 = nn.ConvTranspose2d(32, 16, kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(16, img_channels, kernel_size=3, stride=1, padding=1)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

    def forward(self, z):
        x = self.relu(self.fc2(z))           # Output: (batch_size, 128)
        x = x.view(x.size(0), 32, 2, 2)      # Reshape to (batch_size, 32, 2, 2)
        x = self.upsample(x)                 # Upsample to (batch_size, 32, 4, 4)
        x = self.relu(self.deconv1(x))       # Output: (batch_size, 16, 4, 4)
        x = self.upsample(x)                 # Upsample to (batch_size, 16, 8, 8)
        x = self.sigmoid(self.deconv2(x))    # Output: (batch_size, img_channels, 8, 8)
        return x

class MAE(nn.Module):
    def __init__(self, img_channels=1, feature_dim=32, latent_dim=2):
        super(MAE, self).__init__()
        self.encoder = Encoder(img_channels, feature_dim, latent_dim)
        self.decoder = Decoder(img_channels, feature_dim)

    def mask_input(self, x, mask_ratio=0.25):
        # Generate a mask with 0s and 1s, keeping only (1-mask_ratio) of the original input
        mask = torch.rand(x.shape, device=x.device) > mask_ratio
        x_masked = x * mask
        return x_masked, mask

    def forward(self, x, mask_ratio=0.25):
        x_masked, mask = self.mask_input(x, mask_ratio)  # Apply masking to input
        z = self.encoder(x_masked)
        reconstructed = self.decoder(z)
        return reconstructed, mask

def mae_loss_function(reconstructed, original, mask):
    # Only calculate reconstruction loss on the masked parts
    masked_original = original * mask
    reconstruction_loss = F.mse_loss(reconstructed, masked_original, reduction='sum')
    return reconstruction_loss

def extract_latent_features(model, data_loader, device='cuda'):
    model.eval() 
    latent_features = []  

    with torch.no_grad():
        for batch in tqdm(data_loader, total=len(data_loader), desc="Extracting features"):
            if len(batch) == 2:
                data, _ = batch  
            else:
                (data,) = batch  
            
            data = data.to(device)

            latent_feature = model.encoder(data)
            latent_features.append(latent_feature.cpu().numpy())

    latent_features = np.concatenate(latent_features, axis=0)
    
    return latent_features

## Creation functions

In [6]:
def create_phi(normal_data, c):
    """
    Concept creation function for normal data.
    Uses k-Means clustering to partition normal data into c clusters.
    
    Args:
        normal_data (numpy array): The normal data points.
        c (int): Number of desired normal concepts.
    
    Returns:
        list of numpy arrays: List of normal clusters.
    """
    kmeans = KMeans(n_clusters=c, random_state=42)
    labels = kmeans.fit_predict(normal_data)
    
    normal_concepts = [normal_data[labels == i] for i in range(c)]
    print("Finished creating normal concepts")
    
    return normal_concepts


def create_gamma(anomaly_data, c):
    """
    Concept creation function for anomaly data.
    Uses k-Means clustering to partition anomaly data into c clusters.
    
    Args:
        anomaly_data (numpy array): The anomaly data points.
        c (int): Number of desired anomaly concepts.
    
    Returns:
        list of numpy arrays: List of anomaly clusters.
    """
    kmeans = KMeans(n_clusters=c, random_state=42)
    labels = kmeans.fit_predict(anomaly_data)
    
    anomaly_concepts = [anomaly_data[labels == i] for i in range(c)]
    print("Finished creating anomaly concepts")
    
    return anomaly_concepts
    
def match_lambda(anomaly_concepts, normal_concepts):
    """
    Matches each normal concept with the closest anomaly concept.
    Uses Euclidean distance to determine the best match.
    
    Args:
        anomaly_concepts (list of numpy arrays): List of anomaly clusters.
        normal_concepts (list of numpy arrays): List of normal clusters.
    
    Returns:
        list of tuples: Pairs of (normal_concept, matched_anomaly_concept)
    """
    pairs = []
    remaining_anomalies = anomaly_concepts.copy()

    for normal_concept in normal_concepts:
        normal_centroid = np.mean(normal_concept, axis=0)
        anomaly_centroids = [np.mean(ac, axis=0) for ac in remaining_anomalies]

        distances = cdist([normal_centroid], anomaly_centroids, metric='euclidean')[0]
        closest_idx = np.argmin(distances)

        pairs.append((normal_concept, remaining_anomalies[closest_idx]))
        remaining_anomalies.pop(closest_idx)

    print("Finished matching concept pairs")
    
    return pairs

## Evaluation metrics

In [7]:
def lifelong_roc_auc(R):
    """
    Computes the Lifelong ROC-AUC metric.
    
    Args:
        R (numpy array): NxN matrix of ROC-AUC scores, where R[i, j] is the model's 
                         performance on concept j after learning concept i.
    
    Returns:
        float: Lifelong ROC-AUC score.
    """
    N = R.shape[0]
    lower_triangular_sum = np.sum(np.tril(R))
    normalization_factor = (N * (N + 1)) / 2

    return lower_triangular_sum / normalization_factor

def BWT(R):
    """
    Computes the Backward Transfer (BWT) score.
    
    Args:
        R (numpy array): NxN results matrix.
    
    Returns:
        float: BWT score.
    """
    N = R.shape[0]
    backward_transfer = 0
    count = 0

    for i in range(1, N):
        for j in range(i):
            backward_transfer += (R[i, j] - R[j, j])
            count += 1

    return backward_transfer / count if count > 0 else 0

def FWT(R):
    """
    Computes the Forward Transfer (FWT) score.
    
    Args:
        R (numpy array): NxN results matrix.
    
    Returns:
        float: FWT score.
    """
    N = R.shape[0]
    forward_transfer = 0
    count = 0

    for i in range(N):
        for j in range(i + 1, N): 
            forward_transfer += R[i, j]
            count += 1

    return forward_transfer / count if count > 0 else 0 

## Helper Functions

In [8]:
def pca_feature_selection(X, k, explained_variance_threshold=0.95):

    pca = PCA()
    pca.fit(X)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(cumulative_variance >= explained_variance_threshold) + 1

    pca = PCA(n_components=n_components)
    pca.fit(X)

    feature_importance = np.abs(pca.components_).sum(axis=0)
    top_k_indices = np.argsort(feature_importance)[-k:]
    
    return top_k_indices

def kolmogorov_smirnov_test(X_old, X_new, alpha=0.05):
    p_values = [ks_2samp(X_old[:, i], X_new[:, i]).pvalue for i in range(X_old.shape[1])]
    return np.any(np.array(p_values) < alpha)

def histogram_binning(X, bins=25):
    return np.array([np.histogram(X[:, i], bins=bins, density=True)[0] for i in range(X.shape[1])]).T

def weighted_histogram_binning(X, weights, bins=25):
    histograms = []
    for i in range(X.shape[1]):
        hist, _ = np.histogram(X[:, i], bins=bins, density=True, weights=weights)
        histograms.append(hist)
    return np.array(histograms).T  # shape: (bins, n_features)

def kl_divergence(P, Q):
    P, Q = np.clip(P, 1e-10, None), np.clip(Q, 1e-10, None)  # Avoid log(0)
    return np.sum(P * np.log(P / Q))

def strategic_sample_selection(X_old, X_new, top_k=100, learning_rate=0.01, num_iterations=100):
    H_old, H_new = histogram_binning(X_old), histogram_binning(X_new)
    m_n = np.random.rand(H_new.shape[0])  

    def loss_function(m_n):
        weighted_H_new = H_new * m_n[:, np.newaxis]  
        combined_H = (H_old + weighted_H_new) / 2 
        return kl_divergence(H_new, combined_H) 

    progress_bar = tqdm(total=num_iterations, desc="Optimizing Sample Selection", position=0, leave=True)

    def callback(xk):
        progress_bar.update(1)  

    result = minimize(loss_function, m_n, method="L-BFGS-B", bounds=[(0, 1)] * len(m_n), 
                      options={"maxiter": num_iterations, "ftol": 1e-10}, callback=callback)

    progress_bar.close()

    selected_indices = np.argsort(result.x)[-top_k:]

    return X_new[selected_indices] 

def strategic_forgetting_update_opt(memory_buffer, X_new, drop_k=100, bins=25, num_iterations=100):
    m0 = np.random.uniform(0.5, 1.0, size=memory_buffer.shape[0])
    
    H_new = histogram_binning(X_new, bins=bins)
    H_new = H_new / (np.sum(H_new, axis=0, keepdims=True) + 1e-10)
    
    def loss_function(m):
        H_weighted = weighted_histogram_binning(memory_buffer, m, bins=bins)
        H_weighted = H_weighted / (np.sum(H_weighted, axis=0, keepdims=True) + 1e-10)
        loss = 0
        for j in range(H_new.shape[1]):
            loss += kl_divergence(H_new[:, j], H_weighted[:, j])
        return loss

    progress_bar = tqdm(total=num_iterations, desc="Optimizing Forgetting", position=0, leave=True)

    def callback(xk):
        progress_bar.update(1)

    result = minimize(loss_function, m0, method="L-BFGS-B", bounds=[(0, 1)] * len(m0), 
                      options={"maxiter": num_iterations, "ftol": 1e-10}, callback=callback)
    
    progress_bar.close()
    
    optimized_m = result.x
    print(f"Optimized loss: {result.fun:.5f}")
    
    non_rep_indices = np.where(optimized_m < 0.5)[0]
    
    if len(non_rep_indices) < drop_k:
        additional_needed = drop_k - len(non_rep_indices)
        remaining_indices = np.setdiff1d(np.arange(memory_buffer.shape[0]), non_rep_indices)
        additional_indices = remaining_indices[np.argsort(optimized_m[remaining_indices])[:additional_needed]]
        drop_indices = np.concatenate([non_rep_indices, additional_indices])
    else:
        drop_indices = non_rep_indices[np.argsort(optimized_m[non_rep_indices])[:drop_k]]
    
    updated_buffer = np.delete(memory_buffer, drop_indices, axis=0)
    print(f"Strategic Forgetting: Dropped {len(drop_indices)} samples based on the optimized mask.")
    return updated_buffer


def update_memory_buffer(X_old, X_new_selected, memory_size=3000):
    """Updates memory buffer using strategic forgetting."""
    updated_buffer = np.vstack((X_old, X_new_selected))  

    if updated_buffer.shape[0] > memory_size:
        updated_buffer = updated_buffer[-memory_size:]

    return updated_buffer

## Scenario Design + Evaluation Protocol

In [25]:
def scenario_design(normal_data, anomaly_data, c):
    normal_concepts = create_phi(normal_data, c)
    anomaly_concepts = create_gamma(anomaly_data, c)
    
    scenario = match_lambda(anomaly_concepts, normal_concepts)
    
    return scenario

def evaluation_protocol(model, strategy="naive", replay_buffer_size=3000, memory_size=3000, alpha=0.05, forgetting_quota=100):

    if strategy in ["cumulative"]:
        cumulative_data = []
    
    if strategy in ["replay"]:
        replay_buffer = []

    if strategy == "SSF":
        memory_buffer = None 

    # PREPROCESS
    X = np.vstack((X_train, X_test))
    y = np.hstack((y_train, y_test))
    y = np.where(y == 11, 0, 1)

    num_concepts = 5
    X_normal = X[y == 0]  
    X_anomaly = X[y == 1]
    
    normal_concepts = create_phi(X_normal, num_concepts)
    anomaly_concepts = create_gamma(X_anomaly, num_concepts)
    concept_pairs = match_lambda(anomaly_concepts, normal_concepts)

    T = []  
    E = []
    Y_t = []
    Y = []
    for normal, anomaly in concept_pairs:
    
        normal_train, normal_test = train_test_split(normal, test_size=0.3, random_state=42)
        anomaly_train, anomaly_test = train_test_split(anomaly, test_size=0.3, random_state=42)  
    
        T.append((normal_train, anomaly_train))
        E.append((normal_test, anomaly_test))

        y_normal_train = np.zeros(len(normal_train))
        y_anomaly_train = np.ones(len(anomaly_train))
        y_normal_test = np.zeros(len(normal_test))
        y_anomaly_test = np.ones(len(anomaly_test))

        Y_t.append((y_normal_train, y_anomaly_train))
        Y.append((y_normal_test, y_anomaly_test))
    print("Finished creating concepts")
    
    N = len(T)
    R = np.zeros((N, N))  

    for i, Ti in tqdm(enumerate(T), desc=f"Evaluating using {strategy} strategy"):
        # Input Ti is a tuple of training data (normal, anomalies)
        # Split Ti into Ti_a and Ti_n
        Ti_full = np.vstack((Ti[0], Ti[1]))
        Ti_a = Ti[0]
        Ti_n = Ti[1]

        Y_ti = np.hstack((Y_t[i][0], Y_t[i][1]))

        
        # Apply PCA FS
        k = 31
        top_features_indices = pca_feature_selection(Ti_full, k)

        Ti_full_selected = Ti_full[:, top_features_indices]
        Ti_a_selected = Ti_a[:, top_features_indices]
        Ti_n_selected = Ti_n[:, top_features_indices]
        print("Finished PCA FS on Training")

        
        # Apply DeepInsight
        it = ImageTransformer(
            pixels=8,
            feature_extractor='tsne',
            discretization='lsa'
        )

        it.fit(Ti_full_selected)
        Ti_full_images = it.transform(Ti_full_selected, 'pytorch')
        Ti_a_images = it.transform(Ti_a_selected, 'pytorch')
        Ti_n_images = it.transform(Ti_n_selected, 'pytorch')

        print("Finished Converting Training Images")

        
        # Train MAE on Ti_n_DI
        num_samples, channels, img_height, img_width = Ti_full_images.shape
        
        latent_dim = 16

        X_train_tensor = torch.tensor(Ti_n_images, dtype=torch.float32)
        train_dataset = TensorDataset(X_train_tensor)
        batch_size = 32 
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

        mae_model = MAE(img_channels=channels, feature_dim=32, latent_dim=latent_dim).to(device)
        img_channels = X_train_tensor.shape[1]
        
        optimizer = torch.optim.Adam(mae_model.parameters(), lr=1e-3)
        num_epochs = 20
        
        mask_ratio = 0.75
        
        for epoch in range(1, num_epochs + 1):
            mae_model.train()
            train_loss = 0
        
            progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Epoch {epoch}/{num_epochs}', leave=True)
        
            for batch_idx, (data,) in progress_bar:
                data = data.to(device)
        
                optimizer.zero_grad()
        
                reconstructed, mask = mae_model(data, mask_ratio=mask_ratio)
        
                loss = mae_loss_function(reconstructed, data, mask)
                loss.backward()
                optimizer.step()
        
                train_loss += loss.item()
        
            avg_loss = train_loss / len(train_loader.dataset)
            print(f'Epoch {epoch}, Average Training Loss: {avg_loss:.4f}')
        
        mae_model_curr = mae_model
        print("Finished MAE training")
        
        # Use MAE to Extract features
        Ti_n_latent = extract_latent_features(mae_model_curr, train_loader, device)
        
        
        # Apply strategy
        current_model = clone(model)

        # -- NAIVE --
        if strategy == "naive":
            print("using naive")
            current_model.fit(Ti_n_latent)

        # -- CUMULATIVE --
        if strategy == "cumulative":
            print("using cumulative")
            cumulative_data.extend(Ti_n_latent.tolist())
            current_model.fit(np.array(cumulative_data)) 

        # -- REPLAY -- 
        if strategy == "replay":
            print("using replay")
            if replay_buffer:
                combined_data = np.vstack((np.array(replay_buffer), Ti_n_latent))
            else:
                combined_data = Ti_n_latent

            current_model.fit(combined_data)

            replay_buffer.extend(Ti_n_latent.tolist())

            if len(replay_buffer) > replay_buffer_size:
                replay_buffer = replay_buffer[-replay_buffer_size:]
        
        # -- SSF -- 
        if strategy == "SSF":
            if memory_buffer is None:
                memory_buffer = Ti_n_latent[:memory_size]
            else:
                drift_detected = kolmogorov_smirnov_test(memory_buffer, Ti_n_latent, alpha)
                if drift_detected:
                    memory_buffer = strategic_forgetting_update_opt(memory_buffer, Ti_n_latent, drop_k=forgetting_quota)
                    X_new_selected = strategic_sample_selection(memory_buffer, Ti_n_latent, top_k=forgetting_quota)
                    memory_buffer = np.vstack((memory_buffer, X_new_selected))
                else:
                    memory_buffer = np.vstack((memory_buffer, Ti_n_latent))
                    
                if memory_buffer.shape[0] > memory_size:
                    memory_buffer = memory_buffer[-memory_size:]

            print(len(memory_buffer), type(memory_buffer))
            current_model.fit(memory_buffer)

        # Eval
        for j, ((Ej_normal, Ej_anomaly), (y_normal, y_anomaly)) in enumerate(zip(E, Y)):
            test_data = np.vstack((Ej_normal, Ej_anomaly))
            test_labels = np.hstack((y_normal, y_anomaly))  
            
            test_data_selected = test_data[:, top_features_indices] # PCA FS
            test_data_images = it.transform(test_data_selected, 'pytorch') # DeepInsight
            
            X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)
            train_dataset = TensorDataset(X_test_tensor)
            batch_size = 32 
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
            test_dataset = TensorDataset(X_test_tensor, torch.tensor(test_labels, dtype=torch.long))
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            test_latent_features = extract_latent_features(mae_model_curr, test_loader, device)

            scores = -current_model.decision_function(test_latent_features)  
            R[i, j] = roc_auc_score(test_labels, scores)
            
    return R

# Experiments

In [10]:
R_ssf = evaluation_protocol(IsolationForest(n_estimators=20), strategy="SSF", memory_size=5000, alpha=0.05, forgetting_quota=1000)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Finished creating normal concepts
Finished creating anomaly concepts
Finished matching concept pairs
Finished creating concepts


Evaluating using SSF strategy: 0it [00:00, ?it/s]

Finished PCA FS on Training
Finished Converting Training Images


/tmp/ipykernel_156/4181254767.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(Ti_n_images, dtype=torch.float32)


Epoch 1/20:   0%|          | 0/1801 [00:00<?, ?it/s]

...

/tmp/ipykernel_156/4181254767.py:193: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1110 [00:00<?, ?it/s]

/tmp/ipykernel_156/4181254767.py:193: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1112 [00:00<?, ?it/s]

Lifelong ROC-AUC: 0.4153826822750213, BWT: -0.10910972700810265, FWT: 0.5547969572282282


In [12]:
R_ssf = evaluation_protocol(IsolationForest(n_estimators=20), strategy="naive", memory_size=5000, alpha=0.05, forgetting_quota=1000)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Finished creating normal concepts
Finished creating anomaly concepts
Finished matching concept pairs
Finished creating concepts


Evaluating using naive strategy: 0it [00:00, ?it/s]

Finished PCA FS on Training
Finished Converting Training Images


/tmp/ipykernel_156/4181254767.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(Ti_n_images, dtype=torch.float32)


Epoch 1/20:   0%|          | 0/1801 [00:00<?, ?it/s]

...

/tmp/ipykernel_156/4181254767.py:193: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1110 [00:00<?, ?it/s]

/tmp/ipykernel_156/4181254767.py:193: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1112 [00:00<?, ?it/s]

Lifelong ROC-AUC: 0.3430143081054119, BWT: 0.3089209289052378, FWT: 0.657156929358905


In [ ]:
R_ssf = evaluation_protocol(IsolationForest(n_estimators=20), strategy="Cumulative", memory_size=5000, alpha=0.05, forgetting_quota=1000)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

In [ ]:
R_ssf = evaluation_protocol(IsolationForest(n_estimators=20), strategy="Replay", memory_size=5000, alpha=0.05, forgetting_quota=1000)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

## LOF

In [26]:
R_ssf = evaluation_protocol(LocalOutlierFactor(n_neighbors=20, novelty=True, algorithm="ball_tree"), strategy="SSF", memory_size=5000, alpha=0.05, forgetting_quota=1000)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Finished creating normal concepts
Finished creating anomaly concepts
Finished matching concept pairs
Finished creating concepts


Evaluating using SSF strategy: 0it [00:00, ?it/s]

Finished PCA FS on Training
Finished Converting Training Images


/tmp/ipykernel_156/3232019588.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(Ti_n_images, dtype=torch.float32)


Epoch 1/20:   0%|          | 0/1801 [00:00<?, ?it/s]

...

/tmp/ipykernel_156/3232019588.py:195: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1110 [00:00<?, ?it/s]

/tmp/ipykernel_156/3232019588.py:195: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1112 [00:00<?, ?it/s]

Lifelong ROC-AUC: 0.41826004727608534, BWT: 0.42463619386584445, FWT: 0.5485172811915524


In [28]:
R_naive = evaluation_protocol(LocalOutlierFactor(n_neighbors=20, novelty=True, algorithm="ball_tree"), strategy="naive")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Finished creating normal concepts
Finished creating anomaly concepts
Finished matching concept pairs
Finished creating concepts


Evaluating using naive strategy: 0it [00:00, ?it/s]

Finished PCA FS on Training
Finished Converting Training Images


/tmp/ipykernel_156/3232019588.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(Ti_n_images, dtype=torch.float32)


Epoch 1/20:   0%|          | 0/1801 [00:00<?, ?it/s]

...

/tmp/ipykernel_156/3232019588.py:195: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1110 [00:00<?, ?it/s]

/tmp/ipykernel_156/3232019588.py:195: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1112 [00:00<?, ?it/s]

Lifelong ROC-AUC: 0.33115695571956466, BWT: 0.3237739044808404, FWT: 0.47893226562335245


In [29]:
R_cumulative = evaluation_protocol(LocalOutlierFactor(n_neighbors=20, novelty=True, algorithm="ball_tree"), strategy="cumulative")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_cumulative)}, BWT: {BWT(R_cumulative)}, FWT: {FWT(R_cumulative)}")

Finished creating normal concepts
Finished creating anomaly concepts
Finished matching concept pairs
Finished creating concepts


Evaluating using cumulative strategy: 0it [00:00, ?it/s]

Finished PCA FS on Training
Finished Converting Training Images


/tmp/ipykernel_156/3232019588.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(Ti_n_images, dtype=torch.float32)


Epoch 1/20:   0%|          | 0/1801 [00:00<?, ?it/s]

...

/tmp/ipykernel_156/3232019588.py:195: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1110 [00:00<?, ?it/s]

/tmp/ipykernel_156/3232019588.py:195: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1112 [00:00<?, ?it/s]

Lifelong ROC-AUC: 0.40049750108642346, BWT: 0.4163762621315314, FWT: 0.5937886405869047


In [30]:
R_replay = evaluation_protocol(LocalOutlierFactor(n_neighbors=20, novelty=True, algorithm="ball_tree"), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Finished creating normal concepts
Finished creating anomaly concepts
Finished matching concept pairs
Finished creating concepts


Evaluating using replay strategy: 0it [00:00, ?it/s]

Finished PCA FS on Training
Finished Converting Training Images


/tmp/ipykernel_156/3232019588.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(Ti_n_images, dtype=torch.float32)


Epoch 1/20:   0%|          | 0/1801 [00:00<?, ?it/s]

...

/tmp/ipykernel_156/3232019588.py:195: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1110 [00:00<?, ?it/s]

/tmp/ipykernel_156/3232019588.py:195: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(test_data_images, dtype=torch.float32)


Extracting features:   0%|          | 0/1112 [00:00<?, ?it/s]

Lifelong ROC-AUC: 0.40983416818013024, BWT: 0.434005723874174, FWT: 0.5127006565467802
